In [1]:
import pandas as pd
import numpy as np
from datetime import date, timedelta
import os
import sys
import time
import matplotlib.pyplot as plt
import re
from pathlib import Path
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression,LogisticRegression
from sklearn.model_selection import GridSearchCV,StratifiedKFold,KFold
from sklearn.metrics import accuracy_score,root_mean_squared_error,mean_absolute_error,mean_absolute_percentage_error,f1_score,roc_auc_score,RocCurveDisplay,precision_recall_curve,precision_score,balanced_accuracy_score,classification_report,confusion_matrix,ConfusionMatrixDisplay
from sklearn.tree import DecisionTreeClassifier,DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import roc_auc_score, roc_curve, auc, precision_recall_curve,root_mean_squared_error,r2_score, mean_absolute_error
from sklearn.preprocessing import label_binarize
import warnings
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import SVC,SVR
from xgboost import XGBClassifier,XGBRegressor
import pickle
from sklearn.model_selection import train_test_split



In [2]:
data=pd.read_csv("sed_act_3.26.2025_49_feature.csv")

In [ ]:
tired_level=pd.read_csv(r"C:\Users\Mia\OneDrive - Wright State University\DAGSI Flow\flow_cat_models\tired_levelfeat_6.24.25\feat_12\feat_12_occor\selected_features.csv")
tired_level10=list(tired_level['Feature'].value_counts().head(10).index)
tired_level5=list(tired_level['Feature'].value_counts().head().index)
tired_level5
tired_level6=list(list(tired_level['Feature'].value_counts().head(6).index))
tired_level7=list(tired_level['Feature'].value_counts().head(7).index)
tired_level8=list(tired_level['Feature'].value_counts().head(8).index)
tired_level9=list(tired_level['Feature'].value_counts().head(9).index)
tired_level3=list(list(tired_level['Feature'].value_counts().head(3).index))
tired_level2=list(list(tired_level['Feature'].value_counts().head(2).index))
tired_levelbottom=list(list(tired_level['Feature'].value_counts().tail(2).index))
tired_level1=list(list(heart_rate['Feature'].value_counts().head(1).index))


In [4]:
data.head(1)

time_stamp  tired level  heart_rate  spo2  Stress_level  \
0  2025-01-19 21:59:06+00:00            1          87    95            32   

                 time_window  heart_rate_mean  heart_rate_med  \
0  2025-01-19 21:40:00+00:00        87.115385            88.0   

   heart_rate_sdev  heart_rate_kurt  ...  Stress_level_sdev  \
0         3.072036         2.083275  ...           3.614534   

   Stress_level_kurt  Stress_level_skew  Stress_level_peak_to_peak  \
0          -0.299013          -1.151398                         10   

   Stress_level_mean_derivative  Stress_level_mean_pp_diff  \
0                      0.019608                   0.372549   

   Stress_level_std_pp_diff  Stress_level_peak_to_rms  \
0                  1.552862                  1.068598   

   Stress_level_num_of_peaks  Flow_score_mean  
0                          0              4.0  

[1 rows x 51 columns]

In [3]:
data.describe()

tired level     heart_rate           spo2   Stress_level  \
count  185451.000000  185451.000000  185451.000000  185451.000000   
mean        1.618185      81.693396      97.483988      51.273366   
std         0.971629      13.687171       1.520386      26.737530   
min         1.000000      -4.000000      89.000000      13.000000   
25%         1.000000      71.000000      97.000000      27.000000   
50%         1.000000      80.000000      98.000000      45.000000   
75%         2.000000      90.000000      98.000000      75.000000   
max         5.000000     155.000000     100.000000      97.000000   

       heart_rate_mean  heart_rate_med  heart_rate_sdev  heart_rate_kurt  \
count    185451.000000   185451.000000    185451.000000    185451.000000   
mean         81.730328       81.321513         5.217238         0.393924   
std          12.367441       12.497152         2.103750         1.422712   
min          60.343396       59.000000         0.517549        -4.000000   
25%          71.971429       71.000000         3.872133        -0.352507   
50%          79.863365       79.500000         4.902071         0.107291   
75%          89.516729       89.000000         6.207467         0.783667   
max         119.829268      125.000000        14.584350         9.499579   

       heart_rate_skew  heart_rate_peak_to_peak  ...  Stress_level_sdev  \
count    185451.000000            185451.000000  ...      185451.000000   
mean          0.366166                26.683987  ...           0.157103   
std           0.669196                10.445171  ...           0.679360   
min          -4.000000                -4.000000  ...           0.000000   
25%           0.044877                19.000000  ...           0.000000   
50%           0.405875                26.000000  ...           0.000000   
75%           0.741051                32.000000  ...           0.000000   
max           2.873690                68.000000  ...          10.343535   

       Stress_level_kurt  Stress_level_skew  Stress_level_peak_to_peak  \
count      185451.000000      185451.000000              185451.000000   
mean            0.769960          -3.488168                   0.433845   
std            18.439253           2.597786                   2.921745   
min            -4.000000         -11.550211                  -4.000000   
25%            -4.000000          -4.000000                   0.000000   
50%            -4.000000          -4.000000                   0.000000   
75%            -4.000000          -4.000000                   0.000000   
max           131.407386           9.752868                  29.000000   

       Stress_level_mean_derivative  Stress_level_mean_pp_diff  \
count                 185451.000000              185451.000000   
mean                      -0.175658                  -0.173890   
std                        0.819425                   0.819842   
min                       -4.000000                  -4.000000   
25%                        0.000000                   0.000000   
50%                        0.000000                   0.000000   
75%                        0.000000                   0.000000   
max                        0.400000                   0.500000   

       Stress_level_std_pp_diff  Stress_level_peak_to_rms  \
count             185451.000000             185451.000000   
mean                  -0.156435                  1.017459   
std                    0.828274                  0.086112   
min                   -4.000000                  1.000000   
25%                    0.000000                  1.000000   
50%                    0.000000                  1.000000   
75%                    0.000000                  1.000000   
max                    1.552862                  1.880259   

       Stress_level_num_of_peaks  Flow_score_mean  
count              185451.000000    185451.000000  
mean                    0.037180         3.686571  
std                     0.189202         0.893856  
min   

In [6]:
nul=data[data['heart_rate']==-4]

In [7]:
nul

time_stamp  tired level  heart_rate  spo2  Stress_level  \
42830  2025-02-04 18:56:47+00:00            5          -4    96            93   
42831  2025-02-04 18:56:48+00:00            5          -4    96            93   
42832  2025-02-04 18:56:49+00:00            5          -4    96            93   
42833  2025-02-04 18:56:50+00:00            5          -4    96            93   
42834  2025-02-04 18:56:51+00:00            5          -4    96            93   
...                          ...          ...         ...   ...           ...   
42899  2025-02-04 18:57:56+00:00            2          -4    96            93   
42900  2025-02-04 18:57:57+00:00            2          -4    96            93   
42901  2025-02-04 18:57:58+00:00            2          -4    96            93   
42902  2025-02-04 18:57:59+00:00            2          -4    96            93   
42903  2025-02-04 18:58:00+00:00            2          -4    96            93   

                     time_window  heart_rate_mean  heart_rate_med  \
42830  2025-02-04 18:40:00+00:00        91.903553            92.0   
42831  2025-02-04 18:40:00+00:00        91.903553            92.0   
42832  2025-02-04 18:40:00+00:00        91.903553            92.0   
42833  2025-02-04 18:40:00+00:00        91.903553            92.0   
42834  2025-02-04 18:40:00+00:00        91.903553            92.0   
...                          ...              ...             ...   
42899  2025-02-04 18:40:00+00:00        91.903553            92.0   
42900  2025-02-04 18:40:00+00:00        91.903553            92.0   
42901  2025-02-04 18:40:00+00:00        91.903553            92.0   
42902  2025-02-04 18:40:00+00:00        91.903553            92.0   
42903  2025-02-04 18:40:00+00:00        91.903553            92.0   

       heart_rate_sdev  heart_rate_kurt  ...  Stress_level_sdev  \
42830          4.59142             -4.0  ...                0.0   
42831          4.59142             -4.0  ...                0.0   
42832          4.59142             -4.0  ...                0.0   
42833          4.59142             -4.0  ...                0.0   
42834          4.59142             -4.0  ...                0.0   
...                ...              ...  ...                ...   
42899          4.59142             -4.0  ...                0.0   
42900          4.59142             -4.0  ...                0.0   
42901          4.59142             -4.0  ...                0.0   
42902          4.59142             -4.0  ...                0.0   
42903          4.59142             -4.0  ...                0.0   

       Stress_level_kurt  Stress_level_skew  Stress_level_peak_to_peak  \
42830               -4.0               -4.0                          0   
42831               -4.0               -4.0                          0   
42832               -4.0               -4.0                          0   
42833               -4.0               -4.0                          0   
42834               -4.0               -4.0                          0   
...                  ...                ...                        ...   
42899               -4.0               -4.0                          0   
42900               -4.0               -4.0                          0   
42901               -4.0               -4.0                          0   
42902               -4.0               -4.0                          0   
42903               -4.0               -4.0                          0   

       Stress_level_mean_derivative  Stress_level_mean_pp_diff  \
42830                           0.0                        0.0   
42831                           0.0                        0.0   
42832                           0.0                        0.0   
42833                           0.0                        0.0   
42834                           0.0                        0.0   
...                             ...                        ...   
42899                           0.0                        0.0  

In [3]:


flow_regress_linear=LinearRegression()
flow_regress_dt=DecisionTreeRegressor(splitter='best',max_depth=9)
flow_regress_xgboost=XGBRegressor()
flow_regress_knn=KNeighborsRegressor(n_neighbors=10,weights='distance')
svm_flow = SVR(kernel='poly',
                   C=1,
                   gamma='scale',
                   degree=4,
                   coef0=1)
Flow_regression_models = {
    'LinearRegression':flow_regress_linear,
    'KNN': flow_regress_knn,
    'decision_tree': flow_regress_dt,
    'SVM': svm_flow,
    'XGBregress':flow_regress_xgboost
}

In [4]:
from sklearn.ensemble import HistGradientBoostingRegressor
Flow_regression_models['QuantileRegressor'] = HistGradientBoostingRegressor(loss='quantile', quantile=0.5)

from sklearn.linear_model import BayesianRidge
Flow_regression_models['BayesianRidge'] = BayesianRidge()
